In [1]:
import pandas as pd


df = pd.read_csv('df.csv')    

def df_sales_merge():
    sales = pd.read_csv('sales/ebook_sales_aggregate.csv', index_col=0)
    sales = sales.groupby('isbn13',as_index=False).agg({'totalrevenue':'sum'}).reset_index()
    sale_title = pd.read_csv('AuthorCrosswalkScraper/All_isbn13_for_sales_by_Author_Agg.csv',index_col=0)
    sale_title = sale_title.rename(columns={
    'Author': 'author',
    'Title' : 'title',
    'PublishedDate': 'datepublished',
    'ISBN13' : 'isbn13'
    })
    sale_title['author'] = sale_title['author'].str.replace("'", "").str.replace("[", "").str.replace("]", "")
    sale_title = sale_title.drop_duplicates()
    #print(sales.columns, sale_title.columns)
    
    salesx = pd.merge(sale_title, sales, on='isbn13', how='outer')
    #salesx = salesx.drop(columns=['datepublished_x'])
    salesx = salesx.rename(columns={'datepublished_y' : 'datepublished',
                                   'title': 'sales_title',
                                   'author': 'sales_author'})

    '''
    
    salesx = pd.merge(sale_title, salesx, on='author')
    salesx = salesx.drop_duplicates()
    salesx = salesx.drop(columns=['datepublished_x'])
    salesx = salesx.rename(columns={'datepublished_y' : 'datepublished'})
    #salex = pd.merge(salex)
    '''
    mf = pd.merge(salesx, df , left_on='isbn13', right_on = 'isbn13', how='outer')
    print(mf['advance_id'].value_counts())
    print(mf.columns)
    mf = mf.groupby(['sales_author','sales_title','isbn13'], as_index=False).agg({'totalrevenue':'sum'})
    mf = mf.loc[mf['totalrevenue'] >0]
    
    #mf = pd.merge(mf, df, left_on=['sales_author','sales_title'], right_on=['author', 'title'])
    return mf
    
    #mf2 = pd.merge(salesx, df, left_on='isbn13_y', right_on='isbn13', how='right')
    
    #mf = pd.merge(salesx, df, left_on='isbn13_x', right_on='isbn13')
    #mf = pd.concat([mf1,mf2])
    '''
    mf = mf.drop(columns=['title','isbn13' ])
    mf = mf.drop_duplicates()
    #print(mf.shape, mf.columns)
    print(mf.columns, mf.head())
        
    kf = pd.merge(mf,sale_title, left_on=['author_x','title_x'], right_on=['author','title'], how='inner')
    print(kf.columns,kf.shape, kf.head())
    kf = kf.groupby(['author_x','title_x','datepublished_x'], as_index=False).agg({'totalrevenue':'sum'})
    #kf = kf.groupby('datepublished_x', as_index=False).agg({'totalrevenue':'sum'})
    '''
    '''
    salesx = pd.merge(sales, sale_title, on='isbn13')
    salesx = salesx.drop(columns=['datepublished_y'])
    salesx = salesx.rename(columns={'datepublished_x' : 'datepublished'})
    print(salesx.columns)
    
    salesxx = pd.merge( salesx,sale_title, on=['author','title'])
    salesxx = salesxx.drop(columns=['datepublished_y'])
    salesxx = salesxx.rename(columns={'datepublished_x' : 'datepublished'})
    print(salesxx.columns)
    
    mf1 = pd.merge(salesxx, df, left_on='isbn13_x', right_on='isbn13')
    mf2 = pd.merge(salesxx, df, left_on='isbn13_y', right_on='isbn13')
    mf = pd.concat([mf1,mf2])
    
    '''
    '''
    sales_x = pd.merge(df, sale_title, on=['isbn13'])
    #sales_x = pd.merge(sales_x, sale_title, on=['author','title'])
    #sales_x = sales_x.drop(columns=['datepublished_y','datepublished'])
    print(sales_x.columns, sales_x.shape)
    mm = pd.merge(sales_x, sale_title, left_on=['author_x','title_x'], right_on=['author','title'], how='inner')
    #mn = pd.merge(mm,sales, on='isbn13')
    mm =mm.drop(columns=['datepublished_y','datepublished'])
    print(mm.columns)
    m1 = pd.merge(mm, sales, left_on=['isbn13_x'] , right_on=['isbn13'] , how='inner')
    m2 = pd.merge(mm, sales, left_on=['isbn13_y'] , right_on=['isbn13'] , how='inner')
    mn = pd.concat([m1,m2])
    mn = mn.drop_duplicates()
    #mm = pd.merge(sales_x, df, on= ['author','title'])
    '''
    return salesx
    
def df_sales_merge_time_series():
    i=0
    sale_title = pd.read_csv('AuthorCrosswalkScraper/All_isbn13_for_sales_by_Author_Agg.csv',index_col=0)
    sale_title = sale_title.rename(columns={
    'Author': 'author',
    'Title' : 'title',
    'PublishedDate': 'datepublished',
    'ISBN13' : 'isbn13'
    })
    sale_title['author'] = sale_title['author'].str.replace("'", "").str.replace("[", "").str.replace("]", "")
    sale_title = sale_title.drop_duplicates()
    for file in os.listdir('sales/printbook/'):
        sales = pd.read_csv(f'sales/printbook/{file}')
        sales = sales.groupby('isbn13',as_index=False).agg({'totalrevenue':'sum'}).reset_index()
        
        
        salesx = pd.merge(sale_title, sales, on='isbn13', how='outer')
        #salesx = salesx.drop(columns=['datepublished_x'])
        salesx = salesx.rename(columns={'datepublished_y' : 'datepublished',
                                       'title': 'sales_title',
                                       'author': 'sales_author'})
        mf = pd.merge(salesx, df , left_on='isbn13', right_on = 'isbn13', how='outer')
        #print(mf['advance_id'].value_counts())
        #print(mf.columns)
        mf = mf.groupby(['sales_author','sales_title','isbn13'], as_index=False).agg({'totalrevenue':'sum'})
        mf = mf.loc[mf['totalrevenue'] >0]
        mf['sales_month_year'] = file[-12:-6]
        mf['sales_month_year'] = pd.to_datetime(mf['sales_month_year'], format='%Y%m')
        if i==0:
            xf = mf
            i+=1
            continue
        xf = pd.concat([xf, mf])
        print(f"{file} - processed")
    #mf = pd.merge(mf, df, left_on=['sales_author','sales_title'], right_on=['author', 'title'])
    return xf

In [7]:
df_sales_merge()

advance_id
160215.0    1
128031.0    1
136221.0    1
129130.0    1
160789.0    1
           ..
155392.0    1
153317.0    1
154164.0    1
167932.0    1
7031.0      1
Name: count, Length: 15799, dtype: int64
Index(['sales_title', 'sales_author', 'datepublished_x', 'isbn13', 'index',
       'totalrevenue', 'Unnamed: 0', 'author', 'title', 'datepublished_y',
       'author_l', 'advance_id', 'Rights Category', 'Genre', 'Date',
       'Author(s)', 'Title', 'Publishers', 'Big Publishing House Affilation',
       'Advance', 'Competition', 'Awards', 'Bestseller', 'Self Publishing',
       'Debut', 'Series', 'All', 'Author_extracted', 'fuzz_ratio',
       'title_extracted'],
      dtype='object')


,sales_author,sales_title,isbn13,totalrevenue
215,"""Barbara OConnor""",Wonderland,9.780374e+12,375619.0
307,"""Bianca DArc""",A Touch of Class,9.781950e+12,580214.0
311,"""Bianca DArc""",Bear Meets Girl,9.781950e+12,1421664.0
318,"""Bianca DArc""",Bounty Hunter Bear,9.781370e+12,1370958.0
325,"""Bianca DArc""",Doubling Down,9.780464e+12,535900.0
...,...,...,...,...
375107,Éric Giacometti,Largo Winch - Tome 21 - L'étoile du matin,9.782800e+12,36526.0
375108,Éric Giacometti,Largo Winch - Tome 22 - Les voiles écarlates,9.791035e+12,7749.0
375365,"أحمد خالد توفيق, دار الشروق",يوتوبيا,9.789771e+12,204390.0
375391,"رضوى عاشور, دار الشروق",ثلاثية غرناطة,9.789771e+12,262484.0
